In [1]:
import numpy as np
from scipy.linalg import expm
import math
from qiskit import IBMQ
from qiskit.tools.jupyter import *

'''
provider = IBMQ.load_account()
armonk = provider.get_backend("ibmq_armonk")
armonk
armonk.configuration()
'''

'\nprovider = IBMQ.load_account()\narmonk = provider.get_backend("ibmq_armonk")\narmonk\narmonk.configuration()\n'

In [2]:
# pauli-matrices initialization 
sx = np.mat('0, 1;1, 0')
sy = np.mat('0, -1j;1j, 0')
sz = np.mat('1, 0;0, -1')

# set target and initial gate --> future: want to make user-input target gate
# currently going from Pauli-X to Hadamard

#U_0 = np.mat('0, 1;1 , 0') # initial gate

U_0 = np.array(np.identity(2,dtype=complex))
U_target = sx
#U_target = 1/math.sqrt(2) * np.mat('1, 1;1, -1') # target gate


qubit_freq = 4.71 # will load this from qiskit soon



In [3]:
class Env(object):
    def __init__(self, dt=0.1): # two values in action space
        super(Env, self).__init__()
        self.n_features = 4 # correlated to neural network code
        self.state = np.array([1,0,0,0])
        self.nstep = 0 
        self.dt=dt

    def reset(self):
        self.state = np.array([1,0,0,0])
        self.nstep = 0 
        return self.state

    def step(self, action, coefficient):
      
        # understand what psi does
        psi = np.array([self.state[0:int(len(self.state) / 2)] + self.state[int(len(self.state) / 2):int(len(self.state))] * 1j])
        psi = psi.T
        psi=np.array(psi)

        H =  qubit_freq*sz/2 + coefficient*sx
        U = expm(-1j * H * self.dt) 
        global U_0
        U_0 = U * U_0

        psi = U_0 * psi

        err = 1-abs(np.trace(U_target.H*U_0)/2)**2 # gate fidelity error calculation
        rwd = 10 * (err<0.5)+100 * (err<0.1)+5000*(err < 10e-3)   

        done =( (err < 10e-3) or self.nstep>=np.pi/self.dt ) 
        self.nstep +=1  

        psi=np.array(psi)
        psi_T = psi.T
        self.state = np.array(psi_T.real.tolist()[0] + psi_T.imag.tolist()[0])

        return self.state, rwd, done, 1-err


In [4]:
net = Env()
net.step(1, .40)

1.9716052785725031


(array([ 0.        ,  0.99287594, -0.01995248,  0.11747025]),
 5110,
 True,
 -0.9716052785725031)